# Python 3.9: Cool New Features for You to Try

The code in this notebook is from [this Real Python article](https://realpython.com/python39-new-features/)

## Proper Time Zone Support

In [ ]:
import datetime as dt

In [ ]:
dt.datetime.now(tz=dt.timezone.utc)

In [ ]:
# A "naive" datetime has no time zone information
dt.datetime.now()

The long-time maintainer of `dateutil`, Paul Ganssle, has joined the core team and helped add a new `zoneinfo` standard library module.

In [ ]:
import zoneinfo as zone

The `zoneinfo` module relies on an IANA time zone database residing on your local computer. It's possible, on Windows in particular, that you don't have any such database or that `zoneinfo` cannot locate it. If you get an error like the following, then `zoneinfo` hasn't been able to locate a time zone database:

```
>>> from zoneinfo import ZoneInfo
>>> ZoneInfo("America/Vancouver")
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ZoneInfoNotFoundError: 'No time zone found with key America/Vancouver'
```

An implementation of the IANA Time Zone Database is available on [PyPI](https://pypi.org) as `tzdata`. One can install it using `pip`:

```
python -m pip install tzdata
```


In [ ]:
# Prints exception on Windows when I had not installed `tzdata`. I'm about to install it in our virtual environment so the error should not occur again
try:
    zone.ZoneInfo('America/Vancouver')
except zone.ZoneInfoNotFoundError as zinfe:
    print(f'ZoneInfoNotFoundError: {zinfe}')

In [ ]:
# After installing `tzdata`, I observed no error
zone.ZoneInfo('America/Vancouver')

In [ ]:
# One creates time-zone-aware time stamps using the `tz`...
dt.datetime.now(tz=zone.ZoneInfo('Europe/Oslo'))

In [ ]:
# ...or the `tzinfo` optional parameter to the `datetime` constructors.
dt.datetime(2020, 10, 5, 3, 9, tzinfo=zone.ZoneInfo('America/Vancouver'))

In [ ]:
# Conversion between time zones
release = dt.datetime(2020, 10, 5, 3, 9, tzinfo=zone.ZoneInfo('America/Vancouver'))
release.astimezone(zone.ZoneInfo('Europe/Oslo'))

### Investigating Time Zones

In [ ]:
# List all available time zones
zone.available_timezones()

In [ ]:
# The number of time zones vary with installation (and, I think, over time).
#
# The [article](https://realpython.com/python39-new-features/) identifies 609 time zone names listed.
len(zone.available_timezones())

[Kiritimati](https://en.wikipedia.org/wiki/Kiritimati), also known as Christmas Island, is currently in the westernmost time zone in the world, UTC+14. That hasn’t always been the case. Before 1995, the island was on the other side of the International Date Line, in UTC-10. In order to move across the date line, Kiritimati completely skipped December 31, 1994.

In [ ]:
tz_kiritimati = zone.ZoneInfo('Pacific/Kiritimati')
ts = dt.datetime(1994, 12, 31, 9, 0, 0, tzinfo=zone.ZoneInfo('UTC'))
ts.astimezone(tz_kiritimati)

In [ ]:
# A unit of one hour is useful in ensuing calculations
hour = dt.timedelta(hours=1)

In [ ]:
(ts + 1 * hour).astimezone(tz_kiritimati)

In [ ]:
# One can also see that the offset from UTC changed
tz_kiritimati.utcoffset(dt.datetime(1994, 12, 30)) / hour

In [ ]:
tz_kiritimati.utcoffset(dt.datetime(1995, 1, 1)) / hour

### Using Best Practices

Heuristics:

- When dealing with **civil times** like the time of a meeting, a train departure, or a concert, store the time in their _native_ time zone.
- When dealing with **time stamps** like computer logs, store these kinds of time stamps in UTC.
- The IANA Time Zone Database changes all the time. If your application in sensitive to time zones, update the database "regularly" (particularly on Windows)
- The IANA time zone names, like 'America/Vancouver', identify a time zone unambiguously; however, they are not well-known. To communicate time zone-aware date times to a user, use regular time zone names. These names are available using the `tzname` method on a time zone object.

In [ ]:
# Random time zone
meeting_time_tz = zone.ZoneInfo(list(zone.available_timezones())[563])
meeting_time_tz

In [ ]:
# The time of a local meeting
meeting_time = dt.datetime(2022, 11, 5, 20, 46, 28, tzinfo=meeting_time_tz)
meeting_time

In [ ]:
# Users may not recognize the time zone, 'Australia/Brisbane', nor the UTC offset
str(meeting_time)

In [ ]:
meeting_time.isoformat()

In [ ]:
f'{meeting_time:%c}'

In [ ]:
# Note that one must provide a time stamp to `tzname()` because the name **changes** over time.
f'{meeting_time:%c} {meeting_time_tz.tzname(meeting_time)}'

In [ ]:
log_time = dt.datetime(2018, 11, 12, 16, 56, 8, tzinfo=zone.ZoneInfo('UTC'))
log_time

In [ ]:
str(log_time)

In [ ]:
log_time.isoformat()